In [7]:
import numpy as np
import pandas as pd
import pandas
import tensorflow as tf
from importlib import reload
import graph; reload(graph)
from w266_common import patched_numpy_io

### Unlabeled Sentiment Data Processing

In [5]:
# unzip sentiment data
import zipfile
with zipfile.ZipFile("data/sentiment140.zip","r") as zip_ref:
    zip_ref.extractall("data/")

In [8]:
##### must unzip and re-run with correct file name. (I had manually changed it)

fullSentimentData = pd.read_csv("data/training.1600000.processed.noemoticon.csv", header=None, encoding = "ISO-8859-1")

In [9]:
fullSentimentData.columns = ["Polarity", "Tweet_ID","Time_Stamp","Query", "Twitter_Handle","Tweet_Content"]

In [10]:
fullSentimentData.head(5)

,Polarity,Tweet_ID,Time_Stamp,Query,Twitter_Handle,Tweet_Content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
reducedSentimentData = fullSentimentData.drop(columns=["Polarity","Time_Stamp","Query", "Twitter_Handle"])
reducedSentimentData["Tweet_Content"] = reducedSentimentData["Tweet_Content"].str.lower()
reducedSentimentData.set_index("Tweet_ID")

,Tweet_Content
Tweet_ID,
1467810369,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1467810672,is upset that he can't update his facebook by ...
1467810917,@kenichan i dived many times for the ball. man...
1467811184,my whole body feels itchy and like its on fire
1467811193,"@nationwideclass no, it's not behaving at all...."
1467811372,@kwesidei not the whole crew
1467811592,need a hug
1467811594,@loltrish hey long time no see! yes.. rains a...
1467811795,@tatiana_k nope they didn't have it


In [12]:
reducedSentimentData.to_csv("data/Unlabeled_Tweets_from_Sentiment_Study.CSV", sep="\t", index=False)

Run the following code while in the project folder in the terminal for preprocessing and encoding

`./tweet2vec_encoder.sh data/sentiment_text_only.csv tweet2vec/tweet2vec/best_model/sentiment data/encoded_data/sentiment `

### Sample code below for concatenating the data and merging datasets
##### Need to update after this point

In [29]:
# LOAD THE DATA
fileName = "data/encoded_data/embeddings_w266.npy"
encodedData = np.load(fileName)
encodedData = pd.DataFrame(data = encodedData, columns = ["x_"+str(i) for i in range(500)])
filename = "data/Labeled_Colorado_Flu_Study_Tweets_AvI_RvN_SvO.csv"
coloradoData = pd.read_csv(filename, sep="\t")
print(encodedData.shape)
allData = pandas.concat([coloradoData, encodedData], axis = 1)
print(allData.shape)


# get all data with labels present for the column we care about
allDataVal = allData.dropna(subset=["Related_Label"])
print(allDataVal.shape)

(5270, 500)
(5270, 505)
(1600000, 2)
(4413, 505)


In [3]:
# Sentiment data text only
filename2 = 'data/Unlabeled_Tweets_from_Sentiment_Study.CSV'
sentimentData = pd.read_csv(filename2, sep="\t")
print(sentimentData.shape)

(1600000, 2)


In [4]:
# Extract Text only for preprocessing and embedding
sentimentData['Tweet_Content'].to_csv('data/sentiment_text_only.csv', header=None, index=False)

In [28]:
sentimentData.head(5)
# Add -1 for unlabeled data note for later

,tweet_id tweet_content
0,"1467810369 @user !url - awww , that's a bummer..."
1,1467810672 is upset that he can't update his f...
2,1467810917 @user i dived many times for the ba...
3,1467811184 my whole body feels itchy and like ...
4,"1467811193 @user no , it's not behaving at all..."


In [24]:
allDataVal

,Tweet_ID,Tweet_Content,Awareness_Label,Related_Label,Self_Label,x_0,x_1,x_2,x_3,x_4,...,x_490,x_491,x_492,x_493,x_494,x_495,x_496,x_497,x_498,x_499
1,5222838706,muh. if i am getting sick and it's not swine f...,0.0,1.0,1.0,-0.174253,0.130687,-1.384781,0.459587,-0.777744,...,0.548753,1.287073,0.312468,-0.465325,2.188294,-0.027273,0.113580,-1.535737,-0.806254,-1.206642
3,5918860304,"getting better,no more piggy flu 4 me,it was n...",0.0,1.0,1.0,2.034387,0.432776,-0.038979,-0.650743,0.067163,...,0.699051,1.544988,0.092756,-1.447382,0.576435,1.077597,-0.316096,1.288783,-1.124577,-0.827067
4,4631607800,@robbsterr yay for man txting you.. in other n...,0.0,0.0,1.0,-0.469187,1.415714,-1.875996,0.901807,0.076518,...,0.744410,0.573868,0.070829,-0.891997,0.930053,-0.876152,0.057401,-0.118646,0.211558,-1.097714
5,6004314210,swine flu has arrived at my daughter's kinderg...,0.0,1.0,0.0,-0.658460,1.165405,-0.449825,-0.973872,-0.883874,...,1.596636,0.630788,-2.392556,-2.039080,0.553627,-1.375080,0.043998,-0.833872,-0.364098,-1.476711
6,5225053106,i think im getting flu soon.,0.0,1.0,1.0,0.300915,-0.538062,-0.130646,1.250716,0.042007,...,0.164920,0.193372,-0.313867,0.543114,1.619272,-0.723616,-0.059287,-0.809263,1.339956,-1.840384
10,5922861314,it was a robitussin nighttime cold & flu kind ...,0.0,1.0,1.0,-0.152115,1.043866,0.631438,-1.297971,-0.434821,...,0.508128,0.355164,-1.860092,0.069368,-0.822949,0.278958,-0.303675,-1.515532,0.339055,-2.659122
14,5222406214,is unsure about all this talk of vaccinations....,1.0,1.0,0.0,-0.185663,0.294188,-1.907565,0.231280,-0.580146,...,2.091116,-0.118888,-0.685229,-0.433458,1.345463,-1.608512,-0.521422,0.533002,-0.990734,0.113849
15,6003319713,getting a little fed up with having (insert ra...,0.0,1.0,1.0,0.028184,0.787010,0.158985,-1.695659,-0.531330,...,0.463776,-0.088787,-1.389725,-0.488418,1.387503,1.878750,-0.063749,0.267827,0.508442,-0.838175
16,5916662502,my flu is not going away!!!!! its not even get...,0.0,1.0,1.0,-0.335556,-0.353130,-0.389785,-0.375977,0.760619,...,0.005190,1.432692,0.100060,0.501878,1.390675,-0.103163,-0.468790,-0.917421,-0.211049,-1.471430
18,5234564007,trying to decide whether or not to get the h1n...,1.0,1.0,1.0,0.407059,1.174753,0.509960,-1.477599,1.125104,...,0.213448,0.899588,-2.295275,-1.375099,1.054478,0.801314,-0.775620,0.664303,-0.275791,-2.583229


In [ ]:
# Concatenate the two datasets together unlabeled encoding and unlabeled content dataset


# combinedData = pandas.concat([, ])